In [1]:
import sys
print(sys.executable)

/opt/homebrew/Cellar/jupyterlab/3.6.3/libexec/bin/python3.11


In [7]:
import tkinter as tk
import heapq

# Constants
CANVAS_WIDTH = 400
CANVAS_HEIGHT = 400
GRID_SIZE = 40
ROBOT_LENGTH = 10

# Create the canvas and draw the grid
root = tk.Tk()
root.title("Occupancy Grid Map")
canvas = tk.Canvas(root, width=CANVAS_WIDTH, height=CANVAS_HEIGHT)
canvas.pack()

for i in range(0, CANVAS_WIDTH, GRID_SIZE):
    canvas.create_line(i, 0, i, CANVAS_HEIGHT, fill="gray")
for j in range(0, CANVAS_HEIGHT, GRID_SIZE):
    canvas.create_line(0, j, CANVAS_WIDTH, j, fill="gray")

# Create the robot
robot = canvas.create_rectangle(
    ROBOT_LENGTH ,
    ROBOT_LENGTH,
    ROBOT_LENGTH + 2 * ROBOT_LENGTH,
    ROBOT_LENGTH + 2 * ROBOT_LENGTH,
    fill="red"
)

# Create the goal
goal = canvas.create_rectangle(
    CANVAS_WIDTH,
    CANVAS_HEIGHT, 
    CANVAS_WIDTH-40,
    CANVAS_HEIGHT-40,
    fill="green"
)

# Create the obstacles
obstacles = set()

def toggle_obstacle(event):
    x = event.x // GRID_SIZE
    y = event.y // GRID_SIZE
    if (x, y) in obstacles:
        obstacles.remove((x, y))
        canvas.delete(f"obstacle_{x}_{y}")
    else:
        obstacles.add((x, y))
        canvas.create_rectangle(
            x * GRID_SIZE,
            y * GRID_SIZE,
            (x + 1) * GRID_SIZE,
            (y + 1) * GRID_SIZE,
            fill="black",
            tags=f"obstacle_{x}_{y}"
        )

canvas.bind("<Button-1>", toggle_obstacle)

# Create the path
path = []
visited = set()

def dfs(x, y):
    if x < 0 or x >= CANVAS_WIDTH // GRID_SIZE or y < 0 or y >= CANVAS_HEIGHT // GRID_SIZE:
        return False
    if (x, y) in obstacles or (x, y) in visited:
        return False
    if x == (CANVAS_WIDTH - GRID_SIZE) // GRID_SIZE and y == (CANVAS_HEIGHT - GRID_SIZE) // GRID_SIZE:
        path.append((x, y))
        return True
    visited.add((x, y))
    if dfs(x + 1, y) or dfs(x - 1, y) or dfs(x, y + 1) or dfs(x, y - 1):
        path.append((x, y))
        return True
    return False

def start_pathfindingD():
    global path, visited
    path = []
    visited = set()
    dfs(0, 0)
    path.reverse()

    for i in range(len(path)):
        x, y = path[i]
        canvas.create_rectangle(
            x * GRID_SIZE,
            y * GRID_SIZE,
            (x + 1) * GRID_SIZE,
            (y + 1) * GRID_SIZE,
            fill="red",
            outline="grey",
            tags=f"path_{i}"
        )

# Create the start button
start_button = tk.Button(root, text="Start Pathfinding dfs", command=start_pathfindingD)
start_button.pack()


# Create the path
path = []
visited = set()

def heuristic(x, y):
    # Compute the Manhattan distance between (x, y) and the goal
    # Manhattan distance is just |x1-x2| + |y1-y2|
    # In this formula x2 and y2 is the bottom right corner of the grid
    return abs(x - (CANVAS_WIDTH - GRID_SIZE) // GRID_SIZE) + abs(y - (CANVAS_HEIGHT - GRID_SIZE) // GRID_SIZE)

def start_pathfinding():
    global path, visited
    path = []
    visited = set()

    # Define the heap for A* and add the starting cell
    heap = [(0, (0, 0))]
    g_values = {(0, 0): 0}
    parent = {}

    while heap:
        # Pop the cell with the lowest f value
        f, current = heapq.heappop(heap)

        if current in visited:
            continue

        visited.add(current)
        x, y = current

        # Check if we have reached the goal
        if x == (CANVAS_WIDTH - GRID_SIZE) // GRID_SIZE and y == (CANVAS_HEIGHT - GRID_SIZE) // GRID_SIZE:
            # Reconstruct the path
            while current in parent:
                path.append(current)
                current = parent[current]
            path.append((0, 0))
            path.reverse()

            for i in range(len(path)):
                x, y = path[i]
                canvas.create_rectangle(
                    x * GRID_SIZE,
                    y * GRID_SIZE,
                    (x + 1) * GRID_SIZE,
                    (y + 1) * GRID_SIZE,
                    fill="red",
                    outline="grey",
                    tags=f"path_{i}"
                )
            return

        # Check the neighbors
        for neighbor_x, neighbor_y in [(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)]:
            if neighbor_x < 0 or neighbor_x >= CANVAS_WIDTH // GRID_SIZE or neighbor_y < 0 or neighbor_y >= CANVAS_HEIGHT // GRID_SIZE:
                continue
            if (neighbor_x, neighbor_y) in obstacles or (neighbor_x, neighbor_y) in visited:
                continue

            # Compute the tentative g value
            tentative_g = g_values[current] + 1

            if (neighbor_x, neighbor_y) not in g_values or tentative_g < g_values[(neighbor_x, neighbor_y)]:
                # Update the g and f values, and add the neighbor to the heap
                g_values[(neighbor_x, neighbor_y)] = tentative_g
                f = tentative_g + heuristic(neighbor_x, neighbor_y)
                heapq.heappush(heap, (f, (neighbor_x, neighbor_y)))
                parent[(neighbor_x, neighbor_y)] = current

    # If we get here, there is no path
    print("No path found")

# Create the start button
start_button = tk.Button(root, text="Start Pathfinding A*", command=start_pathfinding)
start_button.pack()


# Start the main loop
root.mainloop()


In [1]:
import tkinter as tk
import heapq
import random

# Constants
CANVAS_WIDTH = 640
CANVAS_HEIGHT = 640
GRID_SIZE = 40
ROBOT_LENGTH = 10
LANDMARKS = 50

# Create the canvas and draw the grid
root = tk.Tk()
root.title("Occupancy Grid Map")
canvas = tk.Canvas(root, width=CANVAS_WIDTH, height=CANVAS_HEIGHT, bg='blue')
canvas.pack()

for i in range(0, CANVAS_WIDTH, GRID_SIZE):
    canvas.create_line(i, 0, i, CANVAS_HEIGHT, fill="gray")
for j in range(0, CANVAS_HEIGHT, GRID_SIZE):
    canvas.create_line(0, j, CANVAS_WIDTH, j, fill="gray")

# Create the robot
robot = canvas.create_rectangle(
    ROBOT_LENGTH ,
    ROBOT_LENGTH,
    ROBOT_LENGTH + 2 * ROBOT_LENGTH,
    ROBOT_LENGTH + 2 * ROBOT_LENGTH,
    fill="red"
)

# Create the goal
goal = canvas.create_rectangle(
    CANVAS_WIDTH,
    CANVAS_HEIGHT, 
    CANVAS_WIDTH-40,
    CANVAS_HEIGHT-40,
    fill="green"
)

# Create Landmarks 

landMarks = set()

def spawn_landMarks():
    for i in range(LANDMARKS):
        x = random.randint(0,CANVAS_WIDTH - 1) // GRID_SIZE
        y = random.randint(0,CANVAS_HEIGHT - 1) //GRID_SIZE
        if (x,y) in obstacles:
            None
        else:
            landMarks.add((x,y))
            canvas.create_rectangle(
                x * GRID_SIZE,
                y * GRID_SIZE,
                (x + 1) * GRID_SIZE,
                (y + 1) * GRID_SIZE,
                fill="gold",
                tags=f"Landmark_{x}_{y}"
            )
            
landMark_button = tk.Button(root, text="Add Landmarks", command=spawn_landMarks)
landMark_button.pack()

# Create the obstacles
obstacles = set()

def toggle_obstacle(event): 
    x = event.x // GRID_SIZE
    y = event.y // GRID_SIZE
    if (x, y) in obstacles:
        obstacles.remove((x, y))
        canvas.delete(f"obstacle_{x}_{y}")
    else:
        obstacles.add((x, y))
        canvas.create_rectangle(
            x * GRID_SIZE,
            y * GRID_SIZE,
            (x + 1) * GRID_SIZE,
            (y + 1) * GRID_SIZE,
            fill="black",
            tags=f"obstacle_{x}_{y}"
        )

canvas.bind("<Button-1>", toggle_obstacle)

# Create the path
path = []
visited = set()

def dfs(x, y):
    if x < 0 or x >= CANVAS_WIDTH // GRID_SIZE or y < 0 or y >= CANVAS_HEIGHT // GRID_SIZE:
        return False
    if (x, y) in obstacles or (x, y) in visited:
        return False
    if x == (CANVAS_WIDTH - GRID_SIZE) // GRID_SIZE and y == (CANVAS_HEIGHT - GRID_SIZE) // GRID_SIZE:
        path.append((x, y))
        return True
    visited.add((x, y))
    if dfs(x + 1, y) or dfs(x - 1, y) or dfs(x, y + 1) or dfs(x, y - 1):
        path.append((x, y))
        return True
    return False

def start_pathfindingD():
    global path, visited
    path = []
    visited = set()
    dfs(0, 0)
    path.reverse()

    for i in range(len(path)):
        x, y = path[i]
        canvas.create_rectangle(
            x * GRID_SIZE,
            y * GRID_SIZE,
            (x + 1) * GRID_SIZE,
            (y + 1) * GRID_SIZE,
            fill="red",
            outline="grey",
            tags=f"path_{i}"
        )

# Create the start button
start_button = tk.Button(root, text="Start Pathfinding dfs", command=start_pathfindingD)
start_button.pack()


# Create the path
path = []
visited = set()

def heuristic(x, y):
    # Compute the Manhattan distance between (x, y) and the goal
    # Manhattan distance is just |x1-x2| + |y1-y2|
    # In this formula x2 and y2 is the bottom right corner of the grid
    return abs(x - (CANVAS_WIDTH - GRID_SIZE) // GRID_SIZE) + abs(y - (CANVAS_HEIGHT - GRID_SIZE) // GRID_SIZE)

def start_pathfinding():
    global path, visited
    path = []
    visited = set()

    # Define the heap for A* and add the starting cell
    heap = [(0, (0, 0))]
    g_values = {(0, 0): 0}
    parent = {}

    while heap:
        # Pop the cell with the lowest f value
        f, current = heapq.heappop(heap)

        if current in visited:
            continue

        visited.add(current)
        x, y = current

        # Check if we have reached the goal
        if x == (CANVAS_WIDTH - GRID_SIZE) // GRID_SIZE and y == (CANVAS_HEIGHT - GRID_SIZE) // GRID_SIZE:
            # Reconstruct the path
            while current in parent:
                path.append(current)
                current = parent[current]
            path.append((0, 0))
            path.reverse()

            for i in range(len(path)):
                x, y = path[i]
                canvas.create_rectangle(
                    x * GRID_SIZE,
                    y * GRID_SIZE,
                    (x + 1) * GRID_SIZE,
                    (y + 1) * GRID_SIZE,
                    fill="red",
                    outline="grey",
                    tags=f"path_{i}"
                )
            return

        # Check the neighbors
        for neighbor_x, neighbor_y in [(x + 1, y), (x - 1, y), (x, y + 1), (x, y - 1)]:
            if neighbor_x < 0 or neighbor_x >= CANVAS_WIDTH // GRID_SIZE or neighbor_y < 0 or neighbor_y >= CANVAS_HEIGHT // GRID_SIZE:
                continue
            if (neighbor_x, neighbor_y) in obstacles or (neighbor_x, neighbor_y) in visited:
                continue

            # Compute the tentative g value
            tentative_g = g_values[current] + 1

            if (neighbor_x, neighbor_y) not in g_values or tentative_g < g_values[(neighbor_x, neighbor_y)]:
                # Update the g and f values, and add the neighbor to the heap
                g_values[(neighbor_x, neighbor_y)] = tentative_g
                f = tentative_g + heuristic(neighbor_x, neighbor_y)
                heapq.heappush(heap, (f, (neighbor_x, neighbor_y)))
                parent[(neighbor_x, neighbor_y)] = current

    # If we get here, there is no path
    print("No path found")

# Create the start button
start_button = tk.Button(root, text="Start Pathfinding A*", command=start_pathfinding)
start_button.pack()


# Start the main loop
root.mainloop()

# update the commit message
